In [2]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)

### Importación de Librerías

In [3]:
# Obtención de la ruta
actual_dir = os.getcwd()
ext_dir = os.path.dirname(actual_dir)
subdir = '/descarga/data/Municipios/Municipios_Clean/'
dir = ext_dir + subdir

#os.makedirs(dir, exist_ok=True)

In [4]:
df = pd.read_csv(dir + 'Data_Climatic_Municipios_Clean.csv', low_memory=False)
df

,Estado,Municipio,Estado_CVE,Municipio_CVE,Año,Mes,Día,Temp_Superficial,Temp_Superficial_MAX,Temp_Superficial_MIN,Temp_2_Metros,Temp_2_Metros_MAX,Temp_2_Metros_MIN,Temp_2_Metros_Pto_Congelación,Temp_2_Metros_Pto_Húmedo,Presión_Superficial,Velocidad_Viento,Humedad_Relativa,Precipitacion
0,Aguascalientes,El Llano,1,10,2020,1,1,12.91,23.01,6.02,12.88,19.87,7.19,7.03,9.95,78.76,5.12,69.81,13.81
1,Aguascalientes,El Llano,1,10,2020,1,2,11.03,19.21,3.86,10.91,16.86,5.05,6.05,8.48,78.48,6.74,74.31,5.85
2,Aguascalientes,El Llano,1,10,2020,1,3,8.24,21.95,0.26,8.14,18.27,1.51,-1.43,3.36,79.02,3.66,59.56,0.02
3,Aguascalientes,El Llano,1,10,2020,1,4,6.03,20.34,-3.12,5.65,16.12,-1.79,-6.48,-0.42,79.68,3.70,49.56,0.00
4,Aguascalientes,El Llano,1,10,2020,1,5,6.23,22.62,-4.48,6.79,18.73,-1.85,-7.67,-0.44,79.60,3.02,42.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441195,Ciudad de México,Milpa Alta,9,9,2023,10,27,17.00,24.44,11.62,17.23,22.40,13.17,14.19,15.70,79.28,1.17,83.38,4.88
3441196,Ciudad de México,Milpa Alta,9,9,2023,10,28,17.24,25.63,10.66,17.22,23.24,11.67,14.01,15.62,79.27,1.19,82.75,2.96
3441197,Ciudad de México,Milpa Alta,9,9,2023,10,29,17.36,26.55,11.39,17.51,23.84,12.43,13.53,15.52,79.16,0.81,79.31,1.07
3441198,Ciudad de México,Milpa Alta,9,9,2023,10,30,16.24,24.74,10.30,16.44,22.36,11.41,11.78,14.11,79.20,1.09,76.50,0.08


### Creacion de DataFrame a nivel Estatal inferido de los datos municipales 

In [ ]:
### Crea Dataframe Estatal inferido a partir del Municipal
df_estatal = df.groupby(['Año','Mes','Estado', 'Estado_CVE']).mean().reset_index()
df_estatal.drop(columns=['Municipio', 'Municipio_CVE','Día'], inplace=True)
df_estatal

In [ ]:
columnas_numericas = df_estatal.select_dtypes(include=[np.number]).columns # Seleccionar solo las columnas numéricas
df_estatal[columnas_numericas] = df_estatal[columnas_numericas].round(2) # Redondear las columnas numéricas a dos decimales

print(df_estatal.shape)
#df_estatal1.to_csv('data/Estados/Data_Climatic_Estados_Inferido.csv', index=False)
df_estatal


'sum': Suma de los valores.
'mean': Media aritmética.
'median': Mediana.
'min': Valor mínimo.
'max': Valor máximo.
'count': Número de elementos no nulos.
'nunique': Número de valores únicos.
'first': Primer valor no nulo.
'last': Último valor no nulo.
'std': Desviación estándar.
'var': Varianza.
'prod': Producto de los valores.
'sem': Error estándar de la media.
'mode': Moda.
'size': Número total de elementos.

### Funciones de Agregacion especificas

In [ ]:
# Diccionario de funciones de agregación por columna
aggregation_functions = {
    'Temp_Superficial': 'mean',
    'Temp_Superficial_MAX': 'mean',
    'Temp_Superficial_MIN': 'mean', 
    'Temp_2_Metros': 'mean', 
    'Temp_2_Metros_MAX': 'mean', 
    'Temp_2_Metros_MIN': 'mean',
    'Temp_2_Metros_Pto_Congelación': 'mean', 
    'Temp_2_Metros_Pto_Húmedo': 'mean', 
    'Presión_Superficial': 'mean',
    'Velocidad_Viento': 'mean', 
    'Humedad_Relativa': 'mean', 
    'Flujo_Evapotranspiración': 'mean', 
    'Perfil_Humedad_Suelo': 'mean',
    'Dias_Sin_Nubosidad': 'mean', 
    'Precipitacion': 'mean', 
    'Horas_De_Sol': 'mean', 
    'Insolacion_Mediodia': 'mean'
}

In [ ]:
# Agrupar por Estado y aplicar las funciones de agregación especificadas
df_estatal2 = df.groupby(['Año','Mes','Estado', 'Estado_CVE']).agg(aggregation_functions).reset_index()

columnas_numericas = df_estatal2.select_dtypes(include=[np.number]).columns # Seleccionar solo las columnas numéricas
df_estatal2[columnas_numericas] = df_estatal2[columnas_numericas].round(2) # Redondear las columnas numéricas a dos decimales

#df_estatal.drop(columns=['Día','Municipio_CVE'], inplace=True)
df_estatal